In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


Q2 : What is the average height of the players?

In [ ]:

# Reading the file csv
df_1 = pd.read_csv(f"tennis_data/away_team.csv")
df_2 = pd.read_csv(f"tennis_data/home_team.csv")

# Concatenate two csv files
merge_data = pd.concat([df_1,df_2], ignore_index=True)

# Filtering the available data based on needs and removing duplicate data and nan data in the height column
final_data = merge_data[["match_id", "name", "height"]].drop_duplicates(subset='name').dropna(subset=['height'])

# Calculate the mean height of the players
mean_height = final_data['height'].mean()
'The mean height of the players is '+ str(mean_height) + ' ' + 'm'

Q8 :Is there a difference in the number of double faults based on gender?

In [ ]:
# Reading the file csv
df_away_team = pd.read_csv(f"tennis_data/away_team.csv")
df_home_team = pd.read_csv(f"tennis_data/home_team.csv")
df_period = pd.read_csv('tennis_data/period.csv')

# Concatenate two csv files and filtering on them
players = pd.concat(
    [df_away_team[['match_id','gender']],
     df_home_team[['match_id','gender']]])

# Filtering and finding double faults
df_double_faults =df_period[df_period['statistic_name'] == 'double_faults']

# Merging player and double_faults files and filtering on all sets
df_final = players.merge(df_double_faults[['match_id', 'home_value','away_value','period']], on='match_id')
df_final = df_final[df_final['period'] == 'ALL']

# Removing duplicate games and add 'home_value' and 'away_value' columns together and store in a new column 'sum_double_faults'
df_final.drop_duplicates(subset='match_id',inplace=True)
df_final['sum_double_fault'] = df_final['home_value'] + df_final['away_value']

# Group the data by 'gender' and generate descriptive statistics for 'sum_double_fault'
result = df_final.groupby('gender')['sum_double_fault'].describe()
result

Q12 : What is the average number of games per set in men's matches compared to women's matches?

In [ ]:
# Load and filter match statistics data from period.csv 
df_period = pd.read_csv('tennis_data/period.csv')
df_game= df_period[['match_id', 'period', 'statistic_name', 'home_value', 'away_value']]
df_game = df_game[df_period['statistic_name'] == 'total_won']
df_game = df_game[df_period['period'] == 'ALL']

# Add total_game column by summing home and away values
df_game['total_game'] = df_game['home_value'] + df_game['away_value']

# Merge home and away team data to create unified gender column per match
df_home = pd.read_csv('tennis_data/home_team.csv')[['match_id', 'gender']]
df_away = pd.read_csv('tennis_data/away_team.csv')[['match_id', 'gender']]
df_gender = pd.merge(df_home, df_away , on='match_id', suffixes=('_home', '_away'))
df_gender = df_gender[df_gender['gender_home'] == df_gender['gender_away']]
df_gender['gender'] = df_gender['gender_home']

# Merge game and gender data to create final dataset and remove duplicate matches
df_final = df_game.merge(df_gender[['match_id', 'gender']],on='match_id',how='inner')
df_final.drop_duplicates(subset='match_id',inplace=True)

# Extract and count number of sets (1st, 2nd, 3rd) per match from period data
df_set_1ST = df_period[df_period['period'].str.contains("1ST",case=False,na=False)] \
        .groupby('match_id')['period'].nunique().reset_index(name='num_set')
df_set_2ND = df_period[df_period['period'].str.contains("2ND",case=False,na=False)] \
        .groupby('match_id')['period'].nunique().reset_index(name='num_set')
df_set_3RD = df_period[df_period['period'].str.contains("3RD",case=False,na=False)] \
        .groupby('match_id')['period'].nunique().reset_index(name='num_set')

# Combine and sum set counts from 1st, 2nd, and 3rd sets per match
counts_set = pd.concat([df_set_1ST,df_set_2ND,df_set_3RD])
counts_set = counts_set.groupby('match_id').sum('num_set')

# Merge set counts with match data and calculate average games per set by gender
df_final = df_final.merge(counts_set,on='match_id')
df_final['avg_game_set'] = df_final['total_game'] / df_final['num_set']
result = df_final.groupby('gender')['avg_game_set'].describe()





Q14 : What is the most common type of surface used in tournaments?

In [ ]:
# Read match_id and ground_type columns from tournament.csv to get the court surface information
df_tournament = pd.read_csv('tennis_data/tournament.csv')[['match_id','ground_type']]

# Remove rows with missing ground_type values to ensure valid surface data
df_tournament.dropna(subset='ground_type', inplace=True)

# Count how many matches were played on each ground type
df_ground_type = df_tournament['ground_type'].value_counts().to_frame().reset_index()

# Calculate the percentage of each ground type relative to total matches
df_ground_type['Percentage'] = (df_ground_type['count'] / df_ground_type['count'].sum() * 100).round(2)

df_ground_type




,ground_type,count,Percentage
0,Hardcourt outdoor,16959,48.30
1,Red clay,11856,33.77
2,Hardcourt indoor,4741,13.50
3,Red clay indoor,512,1.46
4,Grass,509,1.45
5,Carpet indoor,276,0.79
6,Synthetic outdoor,226,0.64
7,Green clay,30,0.09
